# Adaptive Wave Model for Option Pricing: Comparison with Black-Scholes

**Student Project**  
Based on: "Adaptive–Wave Alternative for the Black–Scholes Option Pricing Model"  
arXiv:0911.1834v1 [q-fin.PR] 10 Nov 2009

## Project Overview

This notebook implements and compares:
1. Standard Black-Scholes option pricing model
2. Adaptive Nonlinear Schrödinger (NLS) wave model
3. Comparison of option prices and Greeks
4. Parameter sensitivity analysis

## Key Focus Areas

- **Shock-wave NLS solution** (best agreement with Black-Scholes)
- **Soliton NLS solution** (bright soliton)
- **Option Greeks** (Delta, Gamma, Vega, Theta, Rho)
- **Quantitative comparison** with error metrics


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.optimize import minimize_scalar
import pandas as pd

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')


## Part 1: Black-Scholes Model Implementation

Implement the standard Black-Scholes formulas for European call and put options.


In [ ]:
def black_scholes_call(S, K, T, r, sigma, delta=0):
    """
    Black-Scholes European call option price.
    
    Parameters:
    -----------
    S : float or array
        Current stock price
    K : float
        Strike price
    T : float
        Time to maturity
    r : float
        Risk-free interest rate
    sigma : float
        Volatility
    delta : float
        Dividend yield (default 0)
    
    Returns:
    --------
    float or array
        Call option price
    """
    d1 = (np.log(S/K) + (r - delta + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    N_d1 = 0.5 * (1 + erf(d1 / np.sqrt(2)))
    N_d2 = 0.5 * (1 + erf(d2 / np.sqrt(2)))
    
    call_price = S * N_d1 * np.exp(-delta*T) - K * N_d2 * np.exp(-r*T)
    return call_price


def black_scholes_put(S, K, T, r, sigma, delta=0):
    """
    Black-Scholes European put option price.
    """
    d1 = (np.log(S/K) + (r - delta + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    N_d1 = 0.5 * (1 + erf(d1 / np.sqrt(2)))
    N_d2 = 0.5 * (1 + erf(d2 / np.sqrt(2)))
    
    put_price = K * N_d2 * np.exp(-r*T) - S * N_d1 * np.exp(-delta*T)
    return put_price


# Test Black-Scholes implementation
S_test = 100
K_test = 100
T_test = 1.0
r_test = 0.05
sigma_test = 0.3

call_price = black_scholes_call(S_test, K_test, T_test, r_test, sigma_test)
put_price = black_scholes_put(S_test, K_test, T_test, r_test, sigma_test)

print(f"Black-Scholes Call Price: ${call_price:.4f}")
print(f"Black-Scholes Put Price: ${put_price:.4f}")


## Part 2: NLS Wave Model Implementation

Implement the adaptive nonlinear Schrödinger equation solutions.


In [ ]:
def nls_shock_wave(s, t, sigma, beta, k):
    """
    Shock-wave (dark soliton) solution of NLS equation (13).
    
    ψ₂(s,t) = ±√(-σ/β) tanh(s-σkt) exp(i[ks-0.5σt(2+k²)])
    
    Returns probability density |ψ|²
    """
    u = s - sigma * k * t
    amplitude = np.sqrt(-sigma / beta)
    phase = k * s - 0.5 * sigma * t * (2 + k**2)
    
    psi = amplitude * np.tanh(u) * np.exp(1j * phase)
    return np.abs(psi)**2


def nls_soliton(s, t, sigma, beta, k):
    """
    Bright soliton solution of NLS equation (15).
    
    ψ₄(s,t) = ±√(σ/β) sech(s-σkt) exp(i[ks-0.5σt(k²-1)])
    
    Returns probability density |ψ|²
    """
    u = s - sigma * k * t
    amplitude = np.sqrt(sigma / beta)
    phase = k * s - 0.5 * sigma * t * (k**2 - 1)
    
    psi = amplitude * (1.0 / np.cosh(u)) * np.exp(1j * phase)
    return np.abs(psi)**2


# Test NLS solutions
s_range = np.linspace(50, 150, 200)
t_val = 1.0
sigma = 0.3
beta = 0.05  # Simplified: β = r (interest rate)
k = 1.2

pdf_shock = nls_shock_wave(s_range, t_val, sigma, beta, k)
pdf_soliton = nls_soliton(s_range, t_val, sigma, beta, k)

print("NLS solutions calculated successfully")
print(f"Shock-wave PDF range: [{pdf_shock.min():.4f}, {pdf_shock.max():.4f}]")
print(f"Soliton PDF range: [{pdf_soliton.min():.4f}, {pdf_soliton.max():.4f}]")


In [ ]:
# Compare call option prices
S_range = np.linspace(70, 130, 100)
K = 100
T = 1.0
r = 0.05
sigma = 0.3

# Black-Scholes prices
bs_call = black_scholes_call(S_range, K, T, r, sigma)

# NLS model (using shock-wave solution)
# Note: Need to scale/calibrate NLS PDF to option prices
# This is a simplified comparison - full calibration would use optimization
nls_pdf = nls_shock_wave(S_range, T, sigma, r, k=1.2)

# Normalize and scale for comparison (simplified approach)
nls_scaled = nls_pdf / nls_pdf.max() * bs_call.max()

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(S_range, bs_call, 'b-', linewidth=2, label='Black-Scholes')
ax1.plot(S_range, nls_scaled, 'r--', linewidth=2, label='NLS (Shock-wave)')
ax1.set_xlabel('Stock Price (S)', fontsize=12)
ax1.set_ylabel('Call Option Price', fontsize=12)
ax1.set_title('Call Option Price Comparison', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Error analysis
error = np.abs(bs_call - nls_scaled)
mse = np.mean(error**2)
mae = np.mean(error)

ax2.plot(S_range, error, 'g-', linewidth=2)
ax2.set_xlabel('Stock Price (S)', fontsize=12)
ax2.set_ylabel('Absolute Error', fontsize=12)
ax2.set_title(f'Error Analysis (MSE={mse:.4f}, MAE={mae:.4f})', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
def black_scholes_greeks(S, K, T, r, sigma, delta=0):
    """
    Calculate all Black-Scholes Greeks.
    """
    d1 = (np.log(S/K) + (r - delta + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    N_d1 = 0.5 * (1 + erf(d1 / np.sqrt(2)))
    N_d2 = 0.5 * (1 + erf(d2 / np.sqrt(2)))
    n_d1 = (1/np.sqrt(2*np.pi)) * np.exp(-0.5*d1**2)
    
    # Delta
    delta_greek = N_d1 * np.exp(-delta*T)
    
    # Gamma
    gamma = (n_d1 * np.exp(-delta*T)) / (S * sigma * np.sqrt(T))
    
    # Vega
    vega = S * n_d1 * np.exp(-delta*T) * np.sqrt(T) / 100  # Divided by 100 for percentage
    
    # Theta (per day)
    theta = (-(S * n_d1 * sigma * np.exp(-delta*T)) / (2*np.sqrt(T)) 
             - r * K * np.exp(-r*T) * N_d2 
             + delta * S * N_d1 * np.exp(-delta*T)) / 365
    
    # Rho
    rho = K * T * np.exp(-r*T) * N_d2 / 100  # Divided by 100 for percentage
    
    return {
        'delta': delta_greek,
        'gamma': gamma,
        'vega': vega,
        'theta': theta,
        'rho': rho
    }


# Calculate Greeks
S_greeks = np.linspace(80, 120, 100)
bs_greeks = black_scholes_greeks(S_greeks, K, T, r, sigma)

# Plot Greeks
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

greek_names = ['delta', 'gamma', 'vega', 'theta', 'rho']

for idx, name in enumerate(greek_names):
    axes[idx].plot(S_greeks, bs_greeks[name], 'b-', linewidth=2, label='Black-Scholes')
    axes[idx].set_xlabel('Stock Price (S)', fontsize=12)
    axes[idx].set_ylabel(name.title(), fontsize=12)
    axes[idx].set_title(f'{name.title()} vs Stock Price', fontsize=14)
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

axes[5].axis('off')
plt.tight_layout()
plt.show()


## Part 5: Parameter Sensitivity Analysis

Analyze how different parameters affect the NLS model.


In [ ]:
# Volatility sensitivity
sigmas = [0.1, 0.2, 0.3, 0.4, 0.5]
s_sens = np.linspace(80, 120, 200)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

for sigma in sigmas:
    pdf = nls_shock_wave(s_sens, 1.0, sigma, 0.05, 1.2)
    ax1.plot(s_sens, pdf, linewidth=2, label=f'σ = {sigma}')

ax1.set_xlabel('Stock Price (s)', fontsize=12)
ax1.set_ylabel('Probability Density |ψ|²', fontsize=12)
ax1.set_title('Effect of Volatility (σ) on NLS Wave Function', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Market potential sensitivity
betas = [0.01, 0.03, 0.05, 0.07, 0.10]

for beta in betas:
    pdf = nls_shock_wave(s_sens, 1.0, 0.3, beta, 1.2)
    ax2.plot(s_sens, pdf, linewidth=2, label=f'β = {beta}')

ax2.set_xlabel('Stock Price (s)', fontsize=12)
ax2.set_ylabel('Probability Density |ψ|²', fontsize=12)
ax2.set_title('Effect of Market Potential (β) on NLS Wave Function', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Part 6: Discussion and Conclusions

### Key Findings

1. **NLS Model Capabilities:**
   - The shock-wave solution shows good qualitative agreement with Black-Scholes
   - The wave function approach provides a different perspective on option pricing
   - Parameter sensitivity analysis reveals model behavior

2. **Comparison Results:**
   - [Add your quantitative findings here]
   - [Add error metrics and analysis]

3. **Limitations:**
   - Simplified parameter calibration (β = r)
   - Need for proper scaling/normalization
   - Full calibration requires optimization algorithms

### Future Work

- Implement full parameter calibration (Levenberg-Marquardt)
- Explore combined shock-wave + soliton solution
- Extend to stochastic volatility models
- Real-world data validation
